In [2]:
using Plots
using LinearAlgebra


In [ ]:
function gradψ(H::Function, k::Vector{Float64}, d::Float64)
    # Calculate Eigenvectors
    # k = [kx, ky, kz]

    result = []
    
    for i = 1:3
        dk = zeros(3)
        dk[i] = d
        
        # Get the eigenvectors at +/-dk for each component of k
        eigenvecplus = eigenvecs(H(k+dk))

        eigenvecminus = eigenvecs(H(k-dk))
        
        # keep the matrices, subtract, and calc partial derivative
        dPsi = eigenvecplus - eigenvecminus
    
        # Calculate the partial derivative
        dPsiₖ = dPsi / (2*d)
        push!(result, dPsiₖ)
    end
    # Package back into a vector
    return result

In [ ]:
function Aₙgen(H::Function)
    dk2 = 1e-8
    
    function Aₙs(k)
    # Compute the expectation value of grad psi
        
        H = H(k)
        
        eigenvectors = eigenvecs(H)
        
        result = []
        
        nv = size(eigenvectors)[1]
        
        for i = 1:(nv - 1)
            result_i = []
        
            # Compute grad psi
        
            ∇ψ = gradψ(H, k, dk2)
            ∇ψᵢ = []
            for j = 1:3
                push!(∇ψᵢ, ∇ψ[j][:, i])
            end
            
            # Compute psi*
            eigenvec = eigenvectors[:, i]
            eigenbra = eigenvec'
            
            # <ψ | ∇ψ> = [stuff*] * [stuff]ᵀ
            for k = 1:3
                resultk = eigenbra * ∇ψᵢ[k]
                push!(result_i, resultk)
            end
    
            # Multiply by i
            push!(result, result_i * im)
        end
    
        # return a vector of vectors
        return result
        
    return Aₙs(k)
    # return Aₙs(k)

In [ ]:
# This function calculates the berry phases for a given Hamiltonian

# Inputs: H(k), function representing the Hamiltonian
# Outputs: N-dimensional vector of Float64. representing the berry phase for each eigenstate

function berryphase(H::Function, Klist::Vector{String}, Kdict::dict)::Vector
    # Convert Klist to a list of points
    n = 100
    kpts = interpolate(n, Klist, Kdict, 1) # Vector{Vector{Float64}}
    
    # Generate Aₙ using the Aₙgen function
    A = Aₙgen(H)
    
    nE = size(A([0;0;0])[1]
    
    nk = size(kpts)[1]    # returns the number of k vectors!
    
    bp = zeros(nE)
    
    
    for ik = 1:nk
        
        # Calculate dk1
        k1 = kpts[ik]
        k2 = kpts[(ik + 1) % nk]
    
        dk = k2 - k1
        
        Avecs = A(k1)
        
        for j = 1:nE
            Avec = Avecs[j]
            
            # Dot the Vectors
            product = Avec⋅dk
            
            # Integrate
            bp[j] += product
        
        end
    end
        
    #Return a vector of berry phases
    return bp